In [11]:
import tensorflow as tf
import os

In [12]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

code:

In [13]:
import filetype

In [ ]:
dataDir = "data"
imageTypes = ["jpg", "jpeg", "png", "bmp"]